In [1]:
# Paso 0: Importar la libreria Pandas, Numpy y funciones de sklearn para trabajar con el modelo. 

import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# Paso 0: Ajustar el display del Dataframe para poder desplegar el total de columnas del dataset original.

pd.options.display.max_columns = 130
pd.options.display.max_rows = 120

In [3]:
# Paso 0: Aplicar '%ls' para identificar la carpeta en la que estamos trabajando.

%ls

 El volumen de la unidad C es Windows
 El número de serie del volumen es: 788B-24D1

 Directorio de C:\Users\javie\OneDrive\Escritorio\Ironhack\Semana 3\Banking-Risk\data\clean data

02/02/2024  11:26    <DIR>          .
02/02/2024  10:20    <DIR>          ..
02/02/2024  11:23    <DIR>          .ipynb_checkpoints
01/02/2024  16:26           147.594 1.1 - Cleaning dataset.ipynb
01/02/2024  17:07           123.363 1.2 - Analysis.ipynb
31/01/2024  18:41       166.133.370 application_data.csv
02/02/2024  11:25             5.415 Model 1.ipynb
01/02/2024  15:44        51.924.049 risk_df.csv
02/02/2024  11:26            39.399 scoring_1.ipynb
01/02/2024  17:38            59.591 scoring_2.ipynb
02/02/2024  11:23                72 Untitled.ipynb
               8 archivos    218.432.853 bytes
               3 dirs  375.292.747.776 bytes libres


In [4]:
# Paso 1: Crear un nuevo Dataframe aplicando el metodo read_csv().

risk_tg = pd.read_csv('risk_df.csv')

In [5]:
# Paso 2: Desplegar el Dataframe para comprobar la info con la que vamos a cosntruir la clasificación

risk_tg

,id,target,loan_type,gender,own_car,own_realty,children,client_income,credit_amount,loan_annuity,goods_price,client_income_type,education,family_status,housing_situation,car_age,client_ocupation,family_members,region_rating,city_rating,ext_score_1,ext_score_2,ext_score_3,obs_30,def_30,obs_60,def_60,req_hour,req_day,req_week,req_mon,req_qrt,req_year
0,100002,High Risk,Cash loans,M,No,Yes,0,202500.0,406597.5,24700.5,351000.0,Working,Secondary,Single,Ownership,12.06,Laborers,1,2,2,0.08,0.26,0.14,2,2,2,2,0,0,0,0,0,1
1,100003,Low Risk,Cash loans,F,No,No,0,270000.0,1293502.5,35698.5,1129500.0,State servant,Higher education,Married,Ownership,12.06,Core,2,1,1,0.31,0.62,0.51,1,0,1,0,0,0,0,0,0,0
2,100004,Low Risk,Revolving loans,M,Yes,Yes,0,67500.0,135000.0,6750.0,135000.0,Working,Secondary,Single,Ownership,26.00,Laborers,1,2,2,0.50,0.56,0.73,0,0,0,0,0,0,0,0,0,0
3,100006,Low Risk,Cash loans,F,No,Yes,0,135000.0,312682.5,29686.5,297000.0,Working,Secondary,Married,Ownership,12.06,Laborers,2,2,2,0.50,0.65,0.51,2,0,2,0,0,0,0,0,0,2
4,100007,Low Risk,Cash loans,M,No,Yes,0,121500.0,513000.0,21865.5,513000.0,Working,Secondary,Single,Ownership,12.06,Core,1,2,2,0.50,0.32,0.51,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307500,456251,Low Risk,Cash loans,M,No,No,0,157500.0,254700.0,27558.0,225000.0,Working,Secondary,Separated,Living with parents,12.06,Sales,1,1,1,0.15,0.68,0.51,0,0,0,0,0,0,0,0,0,2
307501,456252,Low Risk,Cash loans,F,No,Yes,0,72000.0,269550.0,12001.5,225000.0,Pensioner,Secondary,Widow,Ownership,12.06,Laborers,1,2,2,0.50,0.12,0.51,0,0,0,0,0,0,0,0,0,2
307502,456253,Low Risk,Cash loans,F,No,Yes,0,153000.0,677664.0,29979.0,585000.0,Working,Higher education,Separated,Ownership,12.06,Managers,1,3,3,0.74,0.54,0.22,6,0,6,0,1,0,0,1,0,1
307503,456254,High Risk,Cash loans,F,No,Yes,0,171000.0,370107.0,20205.0,319500.0,Working,Secondary,Married,Ownership,12.06,Laborers,2,2,2,0.50,0.51,0.66,0,0,0,0,0,0,0,0,0,0


In [6]:
# Paso 3: Establecer las condiciones para la nueva clasificación del 'High Risk' profile.

# Condición 1: El 'income' es 3 veces menos que la cantidad del crédito.

condition_1 = risk_tg.client_income < (risk_tg.credit_amount / 3)

# Condición 2: Los scoring externos son mayores que 0.5.

condition_2 = (risk_tg['ext_score_1'] > 0.5) & (risk_tg['ext_score_2'] > 0.5) & (risk_tg['ext_score_3'] > 0.5)

# Condición 3: El número de consultas al Área de Riesgo es mayor de 4 para un año.

condition_3 = risk_tg.req_year > 4

# Condición 4: El cliente vive en una región y ciudad con rating 2.

condition_4 = (risk_tg.region_rating == 2) & (risk_tg.city_rating == 2)

# Condición 5: Si el tipo de préstamo es 'Cash loans'.

condition_5 = risk_tg.loan_type == 'Cash loans'

# Condición 6: El cliente es el dueño de su vivienda.

condition_6 = (risk_tg.housing_situation == 'Ownership') | (risk_tg.housing_situation == 'Living with parents')

# Condición 7: El cliente está trabajando actualmente.

condition_7 = risk_tg.client_income_type == 'Working'

# Condición 8: El cliente tiene estado civil 'casado'.

condition_8 = risk_tg.family_status == 'Married'

# Condición 9: El cliente tiene educación Secundaria.

condition_9 = risk_tg.education == 'Secondary'

# Condición 10: El cliente no tiene hijos.

condition_10 = risk_tg.children == 0

In [7]:
# Paso 4: Realizar el primer testeo de esta clasificación(10)

# Condiciones para High Risk
condition_1 = risk_tg.client_income < (risk_tg.credit_amount / 3)
condition_2 = (risk_tg['ext_score_1'] > 0.5) & (risk_tg['ext_score_2'] > 0.5) & (risk_tg['ext_score_3'] > 0.5)
condition_3 = risk_tg.req_year > 4
condition_4 = (risk_tg.region_rating == 2) & (risk_tg.city_rating == 2)
condition_5 = risk_tg.loan_type == 'Cash loans'
condition_6 = (risk_tg.housing_situation == 'Ownership') | (risk_tg.housing_situation == 'Living with parents')
condition_7 = risk_tg.client_income_type == 'Working'
condition_8 = risk_tg.family_status == 'Married'
condition_9 = risk_tg.education == 'Secondary'
condition_10 = risk_tg.children == 0

# Asignar valores basados en condiciones
risk_tg['risk_category'] = np.select(
    [
        condition_1,
        condition_2,
        condition_3,
        condition_4,
        condition_5,
        condition_6,
        condition_7,
        condition_8,
        condition_9,
        condition_10
    ],
    ['High Risk'] * 10,  # Valor asignado si la condición es verdadera
    default='Low Risk'   # Valor asignado si ninguna condición es verdadera
)

risk_tg.risk_category.value_counts()

risk_category
High Risk    307502
Low Risk          3
Name: count, dtype: int64

In [8]:
# Paso 5: Realizar el segundo testeo con menos condiciones (8).

# Condiciones para High Risk
condition_1 = risk_tg.client_income < (risk_tg.credit_amount / 3)
condition_2 = (risk_tg['ext_score_1'] > 0.6) & (risk_tg['ext_score_2'] > 0.6) & (risk_tg['ext_score_3'] > 0.6)
condition_3 = risk_tg.req_year > 4
condition_4 = (risk_tg.region_rating == 2) & (risk_tg.city_rating == 2)
condition_5 = risk_tg.loan_type == 'Cash loans'
condition_6 = (risk_tg.housing_situation == 'Ownership') | (risk_tg.housing_situation == 'Living with parents')
condition_7 = risk_tg.client_income_type == 'Working'
condition_8 = risk_tg.family_status == 'Married'


# Asignar valores basados en condiciones
risk_tg['risk_category_2'] = np.select(
    [
        condition_1,
        condition_2,
        condition_3,
        condition_4,
        condition_5,
        condition_6,
        condition_7,
        condition_8,
            ],
    ['High Risk'] * 8,  # Valor asignado si la condición es verdadera
    default='Low Risk'   # Valor asignado si ninguna condición es verdadera
)

risk_tg.risk_category_2.value_counts()

risk_category_2
High Risk    307466
Low Risk         39
Name: count, dtype: int64

In [9]:
# Paso 6: Realizar el tercer testeo con menos condiciones (4).

# Condiciones para High Risk
condition_1 = risk_tg.client_income < (risk_tg.credit_amount / 3)
condition_2 = (risk_tg['ext_score_1'] > 0.6) & (risk_tg['ext_score_2'] > 0.6) & (risk_tg['ext_score_3'] > 0.6)
condition_3 = risk_tg.req_year > 4
condition_4 = (risk_tg.region_rating == 2) & (risk_tg.city_rating == 2)

# Asignar valores basados en condiciones
risk_tg['risk_category_3'] = np.select(
    [
        condition_1,
        condition_2,
        condition_3,
        condition_4,
        
            ],
    ['High Risk'] * 4,  # Valor asignado si la condición es verdadera
    default='Low Risk'   # Valor asignado si ninguna condición es verdadera
)

risk_tg['risk_category_3'].value_counts()

risk_category_3
High Risk    273598
Low Risk      33907
Name: count, dtype: int64

In [10]:
# Paso 7: Realizar el cuarto testeo con menos condiciones (3).

condition_1 = risk_tg.client_income < (risk_tg.credit_amount / 3)
condition_2 = (risk_tg['ext_score_1'] > 0.6) & (risk_tg['ext_score_2'] > 0.6) & (risk_tg['ext_score_3'] > 0.6)
condition_3 = risk_tg.req_year > 4

# Asignar valores basados en condiciones
risk_tg['risk_category_4'] = np.select(
    [
        condition_1,
        condition_2,
        condition_3,
                    ],
    ['High Risk'] * 3,  # Valor asignado si la condición es verdadera
    default='Low Risk'   # Valor asignado si ninguna condición es verdadera
)

risk_tg['risk_category_4'].value_counts()

risk_category_4
High Risk    183367
Low Risk     124138
Name: count, dtype: int64

In [11]:
# Paso 8: Exportar la nueva clasificación a un csv para analizar el modelo.

risk_tg.to_csv('Model_1.csv', index=False)